# Feature Selection using Particle Swarm Optimization (PSO)

The search space for this problem corresponds to the power set of the set of the 54 attributes, which means
$2^{54}$ possibilities. Denote such space by $\mathcal{S}$. The Particle Swarm Optimization (PSO) algorithm can seek the desired subset of attributes without having
to pass through each of the elements in this colossus space. The strategy is to allows solutions (particles) in fixed-size population to move in the solutions space according to the best solution found by itself and to the best solution found
so far by any of the individuals.

Before submitting this problem to the PSO algorithm, it is necessary to define the way solutions (subsets) will
be represented and the function to rank the solutions, also known as the *fitness function*.

## Solutions representation

For this problem, a solution $i$ will be represented by a vector $x^i \in [0,1]^{54}$, in which each component $j$, $x^i_j$, means the probability of selecting the corresponding attribute $j$. Denote $[0,1]^{54}$ by $\mathcal{S_2}$. A threshold $0 \leq \theta \leq 1$ defines 
if the attribute was selected or not: $x^i_j \geq \theta$ means that the attribute $j$ is selected in the solution $i$,
as implemented by the simple function below. In this implementation, $\theta = 0.5$:

In [1]:
def is_selected(xij):
    '''
    Determines if attribute j in solution i was selected
    based on the value on position j of vector x^i.
    '''
    return xij >= 0.5

## Fitness function

This function will rank each solution according to the objective of selecting the most appropriate attribute for this classification problem. Since the purpose is to determine the class of an instance based on the attribute values, the similarity measure of correlation will be applied here. Correlation between two random variables $X$ and $Y$ is given by the
equation:

$$
corr(X,Y) = \frac{cov(X,Y)}{\sigma_X\sigma_Y}
$$

where $cov(X,Y)$ is the covariance between $X$ and $Y$, and $\sigma_X, \sigma_Y$ are the stardard deviations for $X$ and $Y$ respectively. The fitness function is defined as the mean of the unsigned correlations (in $[0,1]$) of each selected attribute with respect to the class. Given that the PSO algorithm used here seeks to minimize the fitness value, the fitness function $f:\mathcal{S_2}\to\mathbb{R}$ is given by the equation:

$$
f(x^i) = 1 - \left(\frac{\sum_{j, x^i_j = 1} |corr(x^i_j,cover\_type)|}{\sum_{j, x^i_j=1} 1}\right).
$$

## Implementation

First of all, it is necessary to load the dataset:

In [2]:
import pandas as pd
# read the dataset
dataset = pd.read_csv("datasets/covertype_norm_train.csv")
# preview
dataset.head()

,elevation,aspect,slope,horiz_dist_hydro,vert_dist_hydro,horiz_dist_road,hillshade_9,hill_shade_noon,hill_shade_15,horiz_dist_fire,...,soil_type_31,soil_type_32,soil_type_33,soil_type_34,soil_type_35,soil_type_36,soil_type_37,soil_type_38,soil_type_39,cover_type
0,-1.929805,1.477831,1.116461,-0.948019,-0.487945,-1.219708,-2.282350,-0.643397,1.322887,-0.934126,...,-0.214972,-0.205073,-0.04331,-0.079247,-0.014425,-0.041672,-0.224338,-0.218305,-0.174891,3
1,1.644997,1.640937,-0.184168,1.692577,0.475531,0.161461,-0.874140,-0.075690,0.761317,-0.947689,...,-0.214972,-0.205073,-0.04331,-0.079247,-0.014425,-0.041672,-0.224338,4.580746,-0.174891,7
2,1.404774,-0.642539,0.998222,-0.182629,-0.096023,0.350281,1.123554,-1.298443,-1.679356,0.476446,...,-0.214972,-0.205073,-0.04331,-0.079247,-0.014425,-0.041672,-0.224338,4.580746,-0.174891,7
3,-0.412357,1.668121,-0.302407,-1.091529,-0.830878,-0.281626,-0.710395,0.011650,0.696520,3.189085,...,-0.214972,-0.205073,-0.04331,-0.079247,-0.014425,-0.041672,-0.224338,-0.218305,-0.174891,2
4,0.429612,1.713428,-1.248319,-0.316572,-0.504275,2.519079,-0.120911,0.535686,0.545328,0.169918,...,-0.214972,-0.205073,-0.04331,-0.079247,-0.014425,-0.041672,-0.224338,-0.218305,-0.174891,5


Correlations with respect to the class are fixed values, and can be calculated using the following (note that possible NaN are removed):

In [3]:
# compute attribute-class correlations
class_correlations = dataset.corr(method="pearson")['cover_type']
# fill possible NaN values with 0
class_correlations.fillna(0, inplace=True)

Now, define the fitness function by:

In [4]:
import numpy as np

def f(x, theta=0.5):
    '''
    Takes a vector in [0,1]^54 and compute its fitness value.
    '''
    selected_attrs = list(map(is_selected, x))
    if (any(selected_attrs)):
        sum_corr = sum([abs(class_correlations[i]) \
                        for i in np.arange(0,dataset.shape[1]-1) if selected_attrs[i]])
        count_attrs = sum(selected_attrs)
        return 1 - (sum_corr/count_attrs)
    else:
        return 1

The PSO implementation used here comes from the `pyswarm` library. An amount of 30 executions is performed:

In [5]:
from pyswarm import pso
import itertools
# define variables's lower bound
lb = np.zeros(dataset.shape[1] - 1)
# define variables' upper bound
ub = np.ones(dataset.shape[1] - 1)
# number of trials
max_trials = 30
# swarm size
swarm_sizes = [25, 50, 100]
# maximum iterations
max_iterations = [50, 100, 200]
# dataframe to store results
results_columns = pd.Index(['swarm_size', 
                            'max_iterations',
                            'fitness']).append(dataset.columns[:-1])
results = pd.DataFrame(columns=results_columns)
# running PSO for max_trials times combining parameters
for swarm_size, max_iterations in itertools.product(*[swarm_sizes, max_iterations]):
    print("swarm_size = ", 
          str(swarm_size),
          ", max_iterations = ",
          str(max_iterations))
    # execute PSO for the selected parameters
    for trial in range(1, max_trials + 1):
        # execute PSO
        xopt, fopt = pso(f, lb, ub, swarmsize=swarm_size, maxiter=max_iterations)
        # booleanize vector
        xopt = list(map(is_selected, xopt))
        # print info
        print("Trial ", str(trial),
              ": ", str(fopt),
              ", selected ", str(sum(xopt)))
        # append result
        results = results.append(pd.Series([swarm_size, 
                                            max_iterations, 
                                            fopt] + xopt,
                                           index=results_columns), 
                                 ignore_index=True)

swarm_size =  25 , max_iterations =  50
Stopping search: maximum iterations reached --> 50
Trial  1 :  0.8883502489828958 , selected  22
Stopping search: maximum iterations reached --> 50
Trial  2 :  0.8666177644230887 , selected  16
Stopping search: maximum iterations reached --> 50
Trial  3 :  0.8786753057920184 , selected  19
Stopping search: maximum iterations reached --> 50
Trial  4 :  0.8821861402688707 , selected  15
Stopping search: maximum iterations reached --> 50
Trial  5 :  0.9006281234234081 , selected  24
Stopping search: maximum iterations reached --> 50
Trial  6 :  0.8474044937174009 , selected  14
Stopping search: maximum iterations reached --> 50
Trial  7 :  0.8931055678806277 , selected  20
Stopping search: maximum iterations reached --> 50
Trial  8 :  0.8825289073936167 , selected  21
Stopping search: maximum iterations reached --> 50
Trial  9 :  0.8870529947744802 , selected  21
Stopping search: maximum iterations reached --> 50
Trial  10 :  0.8970360870555879 , se

Stopping search: maximum iterations reached --> 200
Trial  24 :  0.8846927435743152 , selected  19
Stopping search: maximum iterations reached --> 200
Trial  25 :  0.8740396624694081 , selected  20
Stopping search: maximum iterations reached --> 200
Trial  26 :  0.8933449385072323 , selected  24
Stopping search: maximum iterations reached --> 200
Trial  27 :  0.8858910439180113 , selected  15
Stopping search: maximum iterations reached --> 200
Trial  28 :  0.8950236913964366 , selected  21
Stopping search: maximum iterations reached --> 200
Trial  29 :  0.8921893089667583 , selected  23
Stopping search: maximum iterations reached --> 200
Trial  30 :  0.8870107307509285 , selected  20
swarm_size =  50 , max_iterations =  50
Stopping search: maximum iterations reached --> 50
Trial  1 :  0.8801002322894887 , selected  19
Stopping search: maximum iterations reached --> 50
Trial  2 :  0.8847242669258394 , selected  20
Stopping search: maximum iterations reached --> 50
Trial  3 :  0.87432051

Stopping search: maximum iterations reached --> 200
Trial  17 :  0.8621473136433807 , selected  15
Stopping search: maximum iterations reached --> 200
Trial  18 :  0.8870903463933638 , selected  21
Stopping search: maximum iterations reached --> 200
Trial  19 :  0.8699238886027421 , selected  19
Stopping search: maximum iterations reached --> 200
Trial  20 :  0.8725644665699157 , selected  17
Stopping search: maximum iterations reached --> 200
Trial  21 :  0.8643796053202478 , selected  18
Stopping search: maximum iterations reached --> 200
Trial  22 :  0.8795668045731486 , selected  20
Stopping search: maximum iterations reached --> 200
Trial  23 :  0.8881696898442142 , selected  22
Stopping search: maximum iterations reached --> 200
Trial  24 :  0.8736973037640982 , selected  16
Stopping search: maximum iterations reached --> 200
Trial  25 :  0.8886771408048657 , selected  18
Stopping search: maximum iterations reached --> 200
Trial  26 :  0.8708587842535234 , selected  14
Stopping s

Stopping search: maximum iterations reached --> 200
Trial  10 :  0.8279574068987803 , selected  11
Stopping search: maximum iterations reached --> 200
Trial  11 :  0.8316300276163924 , selected  10
Stopping search: maximum iterations reached --> 200
Trial  12 :  0.880571148447895 , selected  21
Stopping search: maximum iterations reached --> 200
Trial  13 :  0.8772632976639009 , selected  18
Stopping search: maximum iterations reached --> 200
Trial  14 :  0.862245199764142 , selected  13
Stopping search: maximum iterations reached --> 200
Trial  15 :  0.8455341805301687 , selected  12
Stopping search: maximum iterations reached --> 200
Trial  16 :  0.8509845968528442 , selected  13
Stopping search: maximum iterations reached --> 200
Trial  17 :  0.8589625103401117 , selected  15
Stopping search: maximum iterations reached --> 200
Trial  18 :  0.8519464386078313 , selected  13
Stopping search: maximum iterations reached --> 200
Trial  19 :  0.8420528642059293 , selected  11
Stopping sea

Now, check the solution with the least fitness values:

In [6]:
# take a solution with minimum fitness
selected = results.loc[results['fitness'] == results['fitness'].min()]
selected

,swarm_size,max_iterations,fitness,elevation,aspect,slope,horiz_dist_hydro,vert_dist_hydro,horiz_dist_road,hillshade_9,...,soil_type_30,soil_type_31,soil_type_32,soil_type_33,soil_type_34,soil_type_35,soil_type_36,soil_type_37,soil_type_38,soil_type_39
201,100,50,0.812653,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,True,True


Finally, store the results in a CSV for further usage:

In [7]:
# write the results in a csv
results.to_csv('results/pso_selected_attributes.csv', index=False)